# Notebook 7 - Crear y consultar una Base de Datos no relacional orientada a Documentos (Mongo)

## Ejercicio 1: Primeros pasos desde Mongo CLI

En un terminal, verificar si el SGBD Mongo ya está iniciado:

<code>ps -aux | grep mongod</code>

Si no está iniciado, iniciarlo con el comando: <code>mongod</code>

Luego, abrir otra pestaña de terminal (CTRL + Shift + T) e iniciar el cliente CLI de mongo: <code>mongo</code>.

1) Creamos una nueva base de datos llamado <b>nb7-peliculas</b>

<code>use nb7-peliculas</code>

Deberian ver como respuesta del sistema la respuesta: "switched to db nb7-peliculas"

Mongo CLI no utiliza SQL para realizar operaciones CRUD (Create/Read/Update/Delete), utiliza su propia API parecida al lenguaje Javascript.

La variable <code>db</code> representa siempre la base de datos que estamos utilizando (en este caso "nb7-peliculas").

2) Creamos una colección <b>actors</b>

<code>db.actors.insert({firstname:"Will", lastname:"Smith"})</code>

Como pueden verlo, para crear la colección no tuvimos que definir la estructura de la colección, simplemente hemos insertado un documento.

Verificamos que hemos realmente creado la coleccion con:

<code>db.actors.find()</code>

3) Insertamos varios documentos en la colección en un solo comando:

<code>db.actors.insertMany([{firstname:"Scarlett", lastname:"Johanson"},{firstname:"Matt", lastname:"Damon"},{firstname:"Nathan", lastname:"Smith"}])</code>

4) Contamos cuántos documentos tenemos en la colección:

<code>db.actors.count()</code>

Como en SQL, la API de Mongo permite construir consultas para filtrar, ordenar o agrupar datos. Sin embargo, la síntaxis es distinta (y amenudo menos simple e intuitiva que SQL).

5) Filtramos los actores que se llaman 'Smith'

<code>db.actors.find({lastname:"Smith"})</code>

Mongo tambien permite modificar los datos (ojo: Mongo no tiene mecanismo interno para asegurar la coherencia de los datos, es la responsabilidad del programador de la aplicación).

6) Actualizamos un documento. Por ejemplo, "Nathan Smith" se llama ahora "Nathan Lane"

<code>db.actors.update({firstname:"Nathan", lastname:"Smith"},{$set : {lastname:"Lane"}})</code>

Este último comando va a actualizar el primer documento encontrado que corresponde al filtro definido.

7) Supongamos ahora que queremos agregar un campo "gender" en todos los documentos:

¿Qué hace el comando siguiente? ¿Cuál es el limite?

<code>db.actors.update({},{$set : {gender:"M"}})</code>

con este codigo solo modificamos el primer valor y agregamos el campo gender, el resto de actores no tendran el campo gender.


Para actualizar todos los documentos, y no simplemente el primero, podemos utilizar el comando:

<code>db.actors.update({},{$set : {gender:"M"}},{multi:true})</code>

8) Actualizar el documento "Scarlett Johanson", para indicar que es una mujer (gender:"F").

<code>db.actors.update({"firstname":"Scarlett"},{$set : {gender:"F"}})</code>

9) Para suprimir un atributo, podemos utilizar el operador $unset. Por ejemplo:
    
<code>db.actors.update({},{$unset : {gender:1}},{multi:true})</code>

10) Para suprimir un documento completo, podemos utilizar el comando:

<code>db.actors.remove({lastname:"Smith"})</code>

11) Agregamos un atributo de tipo arreglo en un documento

<code>db.actors.update({firstname:"Matt"},{$set : {tab:[1,"a"]}})</code>

12) Añadimos un valor en el arreglo con el operator $push

<code>db.actors.update({firstname:"Matt"},{$push : {tab:["new_value"]}})</code>

¿Cuál es el problema con el último comando? ¿Cuál es la diferencia con el comando siguiente?

<code>db.actors.update({firstname:"Matt"},{$push : {tab:"new_value"}})</code>

En el primer comando tab:["new_value"] es un arreglo, mientras que tab:"new_value" es un valor.

Cómo pueden verlo el SGBD no relacional Mongo es mucho más flexible que MySQL, pero deja mucho más posibilidad de equivocarse.


No duden en consultar la documentación de Mongo para probar otros comandos CRUD: https://docs.mongodb.com/manual/crud/


## Ejercicio 2: Primeras consultas avanzadas con una colección de peliculas

En este segundo ejercicio, trabajaremos con una base de datos existente llamada "movie details", la cual tiene 2295 documentos JSON resumiendo información sobre peliculas.

- Descargar el dataset https://raw.githubusercontent.com/steveren/docs-assets/charts-tutorial/movieDetails.json. ¿Qué contiene este archivo? Un documento JSON? Un arreglo de documentos JSON? Una lista de documentos JSON?1

Contiene un documento Json.

- Importamos los datos en Mongo con el comando siguiente:

<code>mongoimport --db nb7-peliculas --collection movieDetails --drop --file <path/to/movieDetails.json></code>

A continuación, queremos realizar algunas consultas típicas para observar la diferencia entre el lenguaje de consulta SQL y el lenguaje de Mongo.

<b>Nota bena</b>: ¿Qué pasa si agregan la función <code>.pretty()</code> al final de un comando? Por ejemplo: <code>db.actors.find().pretty()</code>

ordena lo que hay en la base de datos no relacional de mongo.

1) Utilizar un comando Mongo para contar cuántos documentos hay en la colección 'movieDetails'. ¿Cuál sería el equivalente en SQL?


mongo
db.movieDetails.count()

sql
SELECT count(campo) FROM tabla

2) Mostrar los títulos (title) y años de producción (year) de todas las peliculas de la colección. ¿A qué clausula SQL correspondría?

<code>db.movieDetails.find({},{"title":1 ,"_id":0,"year":1}).pretty()</code>

si no pongo "_id"=0 automaticamente me muestra en pantalla todos los id

Sql
SHOW TABLES;
o bien
SELECT * FROM sys.tables



3) Contar las peliculas realizadas entre 1990 y 2000

<code>db.movieDetails.count({$and:[{year:{$lte:2000}},{year:{$gte:1950}}]})</code>



4) ¿Cuáles son los titulos de peliculas que ganaron más de 5 awards?

<code> db.movieDetails.find({"awards.wins":{$gte:5}},{title:1,"awards.wins":1,"_id":0}) </code>


5) ¿Cuál es el rating promedio en el sitio imdb para todas las peliculas?

<code>db.movieDetails.aggregate([{$group:{"_id":0, pop:{$avg:"$imdb.rating"}}}])</code>

6) ¿Cuáles son las peliculas en las cuales participó el actor Nicolas Cage?

<code> db.movieDetails.find({"actors":"Nicolas Cage"},{"title":1,"_id":0} ).pretty() </code>

7) ¿Cuál es el número de pelicula por año?

db.movieDetails.aggregate([{"$group":{_id:"$year", count:{$sum:1}}}])


8) ¿Cuál es la pelicula con el mejor rating?


<code> db.movieDetails.find({"imdb.rating":{$gte:9.6}},{title:1,"imdb.rating":0,"_id":0}) </code>



9) ¿Cuáles son los thrillers con un rating de al menos 6?


<code> db.movieDetails.find({"genres":"Thriller", "imdb.rating":{$gte:5}},{"imdb.rating":1,"title":1,"_id":0}).pretty() </code>



10) ¿Cuáles son todos los valores de generos posibles?

 <code>db.movieDetails.distinct("genres").sort() </code>
 
 el sort ordena por alfabeto.
 

11) ¿Cuáles son las peliculas que contienen la palabra 'Star'?


<code> db.movieDetails.find({"title": {$regex:".*Star"}},{title:1,_id:0}) </code>



12) ¿Cuáles son los títulos que empiezan con la letra 'T'?


<code> db.movieDetails.find({ "title": { $regex: "^T" }},{title:1,_id:0}).pretty() </code>

13) ¿Cuáles son las peliculas que no son de tipo Drama o Thriller?


<code>db.movieDetails.find({ "genres":{$nin:["Thriller","Drama"]}},{title:1,_id:0,genres:1}).pretty()</code>


## Ejercicio 3: Integrar Mongo con Python

Desde un entorno Python, conectarse a su base de datos movieDetails y realizar consultas similares al ejercicio 2.

In [24]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017')
db = client['test-database']
courses = db.course 


no supe como hacerlo.

## Ejercicio 4: Conceptos avanzados de Base de Datos: MySQL y Mongo

- <b>Indíce: optimización de consultas</b>

Referencias:
- Indíces en Mongo: https://docs.mongodb.com/manual/indexes/
- Indíces en MySQL: https://dev.mysql.com/doc/refman/8.0/en/optimization-indexes.html
- Introducción a indíces en MySQL: https://www.adictosaltrabajo.com/2015/09/11/introduccion-a-indices-en-mysql/

1) ¿Qué es un Indíce y de qué sirve?

2) ¿Cuándo los SGBDs utilizan los Indíces?

3) ¿Cuáles son los tipos de Indíces en MySQL y Mongo?

4) Si los indíces son utiles, ¿por qué no crear indíces por cada columna/atributo? ¿Cuáles son los límimtes?

5) Conectarse a la base de datos 'sakila' de MySQL y ejecutar la consulta siguiente para poder conservar información sobre el rendimiento de consultas:

<code>set profiling=1;</code>

Ejecutar 10 veces la consulta siguiente:
<code>select * from rental where return_date>="2005-08-25 17:48:44";</code>

Luego, ejecutar el comando:
<code>show profiles;</code>

¿Cuál es la duración promedia del SGBD para responder a esta consulta?

6) Crear un index sobre la columna 'return_date', y medir nuevamente 10 veces el rendimiento de la misma consulta. ¿Cuál es el nuevo valor promedio?

7) ¿Cómo crear índices en Mongo?

- <b>Transacciones</b>

Referencias:
- Introducción a Transacciones en MySQL: http://raknarrok.blogspot.com/2011/03/introduccion-transacciones.html
- Transacción en MySQL: https://dev.mysql.com/doc/refman/8.0/en/commit.html
- Transacción en Mongo: https://docs.mongodb.com/manual/core/transactions/

1) ¿Qué es una transacción? 

2) ¿En términos de transacción, cuáles son las características que deberian garantizar un buen SGBD?

3) En MySQL, crear una tabla <b>bank_user(name VARCHAR(10), account INT)</b> que permite conocer el monto disponible en la cuenta bancaria de algunos usuarios.

4) Realizar una operación que permita transferir 100.000 pesos del usuario A al usuario B. ¿Cómo garantizar que la operación no va a generar algún incoherencia en los datos? Escribir el código SQL correspondiente.

5) Supongamos que queremos hacerlo mismo en Mongo. ¿Mongo soporta transacciones ACID? ¿Desde cuándo? ¿Cómo se puede hacer transacciones? ¿Con qué limitaciones?


- <b>Sharding y Replicación</b>

1) ¿Cuál es el principio del Sharding?

2) ¿Cuál es el principio de la Replicación?

3) ¿MySQL implementa mecanismos de Sharding/Replicación? y Mongo?

Indices

R1: Un índice es un puntero que hace referencia hacia una fila de alguna tabla en la base de datos.

R2: Los SGBDs los utilizan siempre que necesiten realizar busquedas, ya sea para inserción, modificación o eliminación de datos. Estos son usados para realizar esta tarea de una manera mas eficiente por ejemplo al realizar un JOIN o GROUPS BY, etc.

R3: En MySQL existen principalmente PRIMARY KEY, UNIQUE, INDEX, y FULLTEXT, mientras que en mongo vemos incides tales como single-field index, Compound Index, Multikey Index, etc.

R4: Algunas veces no son necesarios, por lo que en estos casos sería un gasto extra de recursos al almacenarlos sin ningun beneficio y tambien traería consigo un aumento en el tiempo de ejecución al realizar acciones de tipo no consulta (ej select).

R5: Aproximadamente es de 0.026 seg.

R6: CREATE INDEX indice ON rental(return_date) USING BTREE;, donde se demora 0.016 seg.

R7:  db.coleccion.ensureIndex({"campo":1}) --> índice simple donde 1 significa si es ascendente (-1 desc).

Transacciones

R1: Son operaciones realizadas de manera que se asegure la consistencia de los datos. Si esto no es garantizado la transacción debe abortarse y volver al estado inicial. Un ejemplo de esto es una transferencia bancaria.

R2: ACID, es decir, atomicidad de varias operaciones, consistencia al crear datos (que no sean inconsistentes), asilamiento (utilizar unicamente conmits) y durabilidad (terminada la transacción los cambios son irreversibles). 

R5: ACID, es decir, atomicidad de varias operaciones, consistencia al crear datos (que no sean inconsistentes), asilamiento (utilizar unicamente conmits) y durabilidad (terminada la transacción los cambios son irreversibles). 



Sharding y Replicación

R1: Consiste en tomar una base de datos y dividirla en partes las cuales serán trabajadas bajos distintos servidores
    
R2: Es el proceso de copiar y mantener actualizados los datos en varios modos de bases de datos.

R3: Los dos mecanismos están implementados en ambos SGBD